In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import re
from tqdm import tqdm
import nltk
import collections
from nltk.stem import WordNetLemmatizer  # For Lemmetization of words
from nltk.corpus import stopwords  # Load list of stopwords
from nltk import word_tokenize # Convert paragraph in tokens



2024-03-22 15:32:10.588309: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 15:32:10.588444: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 15:32:10.767245: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
text_data = pd.read_csv("/kaggle/input/dataset/DataNeuron_Text_Similarity.csv")
print("Shape of text_data : ", text_data.shape)
text_data.head(3)

Shape of text_data :  (3000, 2)


,text1,text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...


In [3]:
text_data.shape

(3000, 2)

In [4]:
text_data.isnull().sum() # Check if text data have any null values

text1    0
text2    0
dtype: int64

In [7]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #Model is imported from this URL
model = hub.load(module_url)
def embed(input):
  return model(input)

In [8]:
message = [text_data['text1'][0], text_data['text2'][0]]
message_embeddings = embed(message)
message_embeddings
type(message_embeddings)

tensorflow.python.framework.ops.EagerTensor

In [9]:
type(message_embeddings[0])

tensorflow.python.framework.ops.EagerTensor

In [10]:
type(tf.make_ndarray(tf.make_tensor_proto(message_embeddings)))

numpy.ndarray

In [12]:
a_np = tf.make_ndarray(tf.make_tensor_proto(message_embeddings))

In [13]:
from numpy import dot                                           # to calculate the dot product of two vectors
from numpy.linalg import norm                                   #for finding the norm of a vector

ans = []                                                        # This list will contain the cosin similarity value for each vector pair present.
for i in range(len(text_data)):
  messages = [text_data['text1'][i], text_data['text2'][i]]               #storing each sentence pair in messages
  message_embeddings = embed(messages)                          #converting the sentence pair to vector pair using the embed() function
  a = tf.make_ndarray(tf.make_tensor_proto(message_embeddings)) #storing the vector in the form of numpy array
  cos_sim = dot(a[0], a[1])/(norm(a[0])*norm(a[1]))             #Finding the cosine between the two vectors
  ans.append(cos_sim)                                           #Appending the values into the ans list

In [14]:
len(ans)

3000

In [15]:
Ans = pd.DataFrame(ans, columns = ['Similarity_Score'])         #converting the ans list into Dataframe so that we can add it to our "Data"

In [16]:
Ans.head()

,Similarity_Score
0,0.272669
1,0.277622
2,0.169011
3,0.157467
4,0.246201


In [20]:
text_data['Similarity_Score'] = text_data['Similarity_Score'] + 1               #adding 1 to each of the values of Similarity_Score to make the values from 0 to 2. (Initially it was from [-1,1])

In [21]:
text_data.head(5)

,text1,text2,Similarity_Score
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,2.272669
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,2.277622
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,2.169011
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,2.157467
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,2.246201


In [22]:
text_data['Similarity_Score'] = text_data['Similarity_Score']/text_data['Similarity_Score'].abs().max() #Normalizing the Similarity_Score to get the value between 0 and 1

In [23]:
Submission = text_data[['text1','text2', 'Similarity_Score']]

In [26]:
Submission.head()

,text1,text2,Similarity_Score
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,0.757556
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,0.759207
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,0.723004
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,0.719156
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,0.748734


#Flask APP